**Jupyter notebook for test of any type**

In [42]:
import sys
import os
import pandas as pd
from datasets import load_dataset
import os
import numpy as np
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc, min, max, to_timestamp, col, count, date_trunc, udf
import langdetect 
from pyspark.sql.types import StringType

# Get the absolute path of the project root
project_root = os.path.abspath("..")  # Adjust if necessary

# Add the src directory to Python's path
sys.path.append(os.path.join(project_root, "src"))

if project_root not in sys.path:
    sys.path.append(project_root)


In [3]:
from data import s3_utils as s3U

In [7]:
df = pd.read_csv("hf://datasets/Shadow-Blade/financialNews/financialNews.csv")

In [ ]:
os.makedirs('../data/raw/', exist_ok=True) 
df.to_csv('../data/raw/financialNews_shadow.csv')

In [3]:
bucket= "financialdata-sa"
news_external = "RawNews/All_external.csv"

In [4]:
news1 = s3U.get_csv_as_spark(bucket_name=bucket, s3_key=news_external)

your 131072x1 screen size is bogus. expect trouble
25/03/14 19:16:39 WARN Utils: Your hostname, Andres-F resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/14 19:16:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/14 19:16:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 19:16:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [ ]:
news1.show(5)

+--------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|                Date|       Article_title|Stock_symbol|                 Url|        Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|
+--------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|2020-06-05 06:30:...|Stocks That Hit 5...|           A|https://www.benzi...|Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-06-03 06:45:...|Stocks That Hit 5...|           A|https://www.benzi...|Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-05-26 00:30:...|71 Biggest Movers...|           A|https://www.benzi...|       Lisa Levin|  NULL|   NULL|       NULL|        NULL|            NULL|   

In [4]:
spark = SparkSession.builder \
    .appName("ConvertCSVtoParquet") \
    .getOrCreate()

df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../data/raw/nasdaq_exteral_data.csv")

your 131072x1 screen size is bogus. expect trouble
25/03/21 10:11:45 WARN Utils: Your hostname, SSMRS3-048E9600 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/21 10:11:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/21 10:11:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_csv.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)



In [9]:
df_csv.show(50)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|          Unnamed: 0|                Date|       Article_title|        Stock_symbol|                 Url|           Publisher|              Author|             Article|         Lsa_summary|        Luhn_summary|    Textrank_summary|Lexrank_summary|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|                 0.0|2023-12-16 23:00:...|Interesting A Put...|                   A|https://www.nasda...|                NULL|                NULL|Investors in Agil...|                NULL|                NULL|                NULL|           NULL|
|The

In [24]:

df_csv_5gb = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .option("quote", "\"") \
    .option("delimiter", ",") \
    .option("mode", "PERMISSIVE") \
    .load("../data/raw/All_external.csv")

In [29]:
df_csv_5gb.describe()

DataFrame[summary: string, Article_title: string, Stock_symbol: string, Url: string, Publisher: string, Author: string, Article: string, Lsa_summary: string, Luhn_summary: string, Textrank_summary: string, Lexrank_summary: string]

In [13]:
df_csv_5gb.show(200)

+--------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+
|                Date|       Article_title|Stock_symbol|                 Url|           Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|
+--------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+
|2020-06-05 06:30:...|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-06-03 06:45:...|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-05-26 00:30:...|71 Biggest Movers...|           A|https://www.benzi...|          Lisa Levin|  NULL|   NULL|       NULL|        NULL|  

In [27]:
df_csv_5gb.sort(asc("Date")).show(60)

+-------------------+--------------------+------------+--------------------+--------------+----------+--------------------+-----------+------------+----------------+---------------+
|               Date|       Article_title|Stock_symbol|                 Url|     Publisher|    Author|             Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|
+-------------------+--------------------+------------+--------------------+--------------+----------+--------------------+-----------+------------+----------------+---------------+
|1914-09-16 00:00:00|1914. Русские вой...|        NULL|https://lenta.ru/...|Первая мировая|Библиотека|Бои у Сопоцкина и...|       NULL|        NULL|            NULL|           NULL|
|1914-09-16 00:00:00|1914. Праздновани...|        NULL|https://lenta.ru/...|Первая мировая|Библиотека|Министерство наро...|       NULL|        NULL|            NULL|           NULL|
|1914-09-17 00:00:00|1914. Das ist Nes...|        NULL|https://lenta.ru/...|Первая мировая

In [31]:
df_5gb = df_csv_5gb.withColumn(
    "Date_parsed", to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss")
)

In [33]:
df_5gb.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)
 |-- Date_parsed: timestamp (nullable = true)



In [36]:
df_5gb.select(min("Date").alias("min_date"), max("Date").alias("max_date")).show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|1914-09-16 00:00:00|2020-06-11 15:12:35|
+-------------------+-------------------+



In [37]:
df_5gb.groupBy("Stock_symbol").agg(count("*").alias("ticker_count")) \
       .orderBy("Stock_symbol", ascending=True) \
       .show(50)  # Show top 50 tickers

+------------+------------+
|Stock_symbol|ticker_count|
+------------+------------+
|        NULL|     9804627|
|           A|        2369|
|          AA|        2953|
|         AAC|         379|
|        AADR|          32|
|         AAL|         494|
|        AAMC|         132|
|        AAME|          87|
|         AAN|        1309|
|        AAOI|         452|
|        AAON|         326|
|         AAP|        2214|
|        AAPL|         473|
|         AAU|          85|
|         AAV|         127|
|        AAVL|         191|
|        AAWW|         322|
|        AAXJ|          76|
|          AB|        1065|
|        ABAC|          72|
|        ABAX|         100|
|         ABB|        1433|
|        ABBV|         753|
|         ABC|         722|
|        ABCB|          20|
|        ABCD|         130|
|        ABCO|         408|
|        ABCW|          37|
|        ABDC|         116|
|        ABEV|         309|
|         ABG|         323|
|        ABGB|         204|
|        ABIO|      

In [39]:
df_5gb.groupby(date_trunc("month", "date").alias("month")) \
       .count() \
       .orderBy("month") \
       .show()

+-------------------+-----+
|              month|count|
+-------------------+-----+
|1914-09-01 00:00:00|    5|
|1969-12-01 00:00:00|    1|
|1999-08-01 00:00:00|   17|
|1999-09-01 00:00:00|  580|
|1999-10-01 00:00:00|  726|
|1999-11-01 00:00:00|  853|
|1999-12-01 00:00:00|  905|
|2000-01-01 00:00:00|  682|
|2000-02-01 00:00:00| 1004|
|2000-03-01 00:00:00| 1226|
|2000-04-01 00:00:00| 1202|
|2000-05-01 00:00:00| 1166|
|2000-06-01 00:00:00| 1283|
|2000-07-01 00:00:00| 1277|
|2000-08-01 00:00:00| 1336|
|2000-09-01 00:00:00| 1590|
|2000-10-01 00:00:00| 1781|
|2000-11-01 00:00:00| 1835|
|2000-12-01 00:00:00| 1794|
|2001-01-01 00:00:00| 1676|
+-------------------+-----+
only showing top 20 rows



In [48]:
df_5gb.show(10)

+-------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+-------------------+
|               Date|       Article_title|Stock_symbol|                 Url|           Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|        Date_parsed|
+-------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+-------------------+
|2020-06-05 08:30:54|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|2020-06-05 08:30:54|
|2020-06-03 08:45:20|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|2020-06-03 08:45:20|
|2020-05-26 02:30:07|71 Biggest Movers...|   

In [50]:
def detect_lang(text):
    try:
        return langdetect.detect(text)
    except:
        return "unknown"

detect_lang_udf = udf(detect_lang, StringType())

df_lang = df_5gb.withColumn("detected_lang", detect_lang_udf(df_5gb["Article_title"]))

df_english_only = df_lang.filter(df_lang["detected_lang"] == "en")

In [53]:
df_english_only.cache()

DataFrame[Date: timestamp, Article_title: string, Stock_symbol: string, Url: string, Publisher: string, Author: string, Article: string, Lsa_summary: string, Luhn_summary: string, Textrank_summary: string, Lexrank_summary: string, Date_parsed: timestamp, detected_lang: string]

In [51]:
df_english_only.show(10)

+-------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+-------------------+-------------+
|               Date|       Article_title|Stock_symbol|                 Url|           Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|        Date_parsed|detected_lang|
+-------------------+--------------------+------------+--------------------+--------------------+------+-------+-----------+------------+----------------+---------------+-------------------+-------------+
|2020-06-05 08:30:54|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|2020-06-05 08:30:54|           en|
|2020-06-03 08:45:20|Stocks That Hit 5...|           A|https://www.benzi...|   Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|2020-06-03

25/03/21 11:50:32 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 41 (TID 502): Attempting to kill Python Worker
